# Claude & AWS Bedrock API Requests

In [1]:
from dotenv import load_dotenv
import boto3
# import json

## Load environment variables

In [ ]:
load_dotenv("../../.env", override=True)

True

## Create Bedrock Runtime Client

In [3]:
client = boto3.client("bedrock-runtime", region_name="us-east-2")

Some models are not available on all regions. To solve the problem of which model is present in which region, we have inference profiles.

## Inference Profile
Automatically routes request to other regions where a specific model is hosted. If it is not in us-east-2 and present in us-west-2, then it will automatically request for it from there.

In [4]:
# we use the inference profile ID for on-demand access
model_id = "us.anthropic.claude-3-5-haiku-20241022-v1:0"

In [5]:
user_message = {
    "role": "user",
    "content": [
        {"text": "What is 1+1?"}
    ]
}

In [6]:
response = client.converse(
    modelId = model_id,
    messages = [user_message]
)

In [8]:
response

{'ResponseMetadata': {'RequestId': '67f0b361-88d3-4766-84ac-efab63819c42',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 21 Aug 2025 16:51:11 GMT',
   'content-type': 'application/json',
   'content-length': '298',
   'connection': 'keep-alive',
   'x-amzn-requestid': '67f0b361-88d3-4766-84ac-efab63819c42'},
  'RetryAttempts': 0},
 'output': {'message': {'role': 'assistant',
   'content': [{'text': '1+1 = 2'}]}},
 'stopReason': 'end_turn',
 'usage': {'inputTokens': 14,
  'outputTokens': 11,
  'totalTokens': 25,
  'cacheReadInputTokens': 0,
  'cacheWriteInputTokens': 0},
 'metrics': {'latencyMs': 898}}

In [9]:
response['output']['message']['content'][0]['text']

'1+1 = 2'

In [10]:
response['output']['message']

{'role': 'assistant', 'content': [{'text': '1+1 = 2'}]}

If we do the following, it would be sent off as a new message without the previous context

```python
response = client.converse(
    modelId = model_id,
    messages = [{
        "role": "user",
        "content": [{
                "text": "And 3 more?"
                }
            ]
        }
    ]
)
```

## Multi-Turn Conversations

### Helper Functions to Maintain Context of Prev Conversations

In [5]:
def add_user_message(messages: list[dict], text: str) -> None:
    user_message = {
        "role": "user",
        "content": [
            {"text": text}
        ]
    }
    messages.append(user_message)

def add_assistant_message(messages: list[dict], text: str) -> None:
    assistant_message = {
        "role": "assistant",
        "content": [
            {"text": text}
        ]
    }
    messages.append(assistant_message)

def chat(messages: list[dict]) -> str:
    response = client.converse(
        modelId = model_id,
        messages = messages
    )
    return response["output"]["message"]["content"][0]["text"]

In [ ]:
# 1. Initialize a list of messages
messages = []

# 2. Add in the initial user message
add_user_message(messages, "What is 1+1?")
print(messages)

[{'role': 'user', 'content': [{'text': 'What is 1+1?'}]}]


In [13]:
# 3. Pass the list of messages into the chat to get an answer
answer = chat(messages)
print(answer)

1+1 = 2.


In [14]:
# 4. Take the answer & add it as an assistant message in our list
add_assistant_message(messages, answer)
print(messages)

[{'role': 'user', 'content': [{'text': 'What is 1+1?'}]}, {'role': 'assistant', 'content': [{'text': '1+1 = 2.'}]}]


In [15]:
# 5. Add the user's follow up question
add_user_message(messages, "And what's 3 more?")
print(messages)

[{'role': 'user', 'content': [{'text': 'What is 1+1?'}]}, {'role': 'assistant', 'content': [{'text': '1+1 = 2.'}]}, {'role': 'user', 'content': [{'text': "And what's 3 more?"}]}]


In [16]:
# 6. Call chat again with the list of messages to get the final answer
answer = chat(messages)
print(answer)

If you add 3 more to 2, the result would be 5.

So, 2 + 3 = 5.


# Simple Chatbot Exercise

In [8]:
# initial list of messages
messages = []


while True:
    # Get user input
    user_input = input("You >> ")

    # Check for exit commands
    if user_input.lower() in ['quit', 'exit', 'bye', 'q']:
        print("Goodbye! Exiting chat...")
        break

    # In VSCode print the input too:
    print("---------------------------------")
    print(f"You >> {user_input}")

    # Add user's input to list of messages
    add_user_message(messages, user_input)

    # Send list of messages to Claude on Bedrock
    answer = chat(messages)

    # Add generated text to list of messages
    add_assistant_message(messages, answer)

    # Print the generated text for the user to see
    print("AI >> "+answer)
    print("---------------xxx---------------")


---------------------------------
You >> In one word what's the capital of Australia?
AI >> Canberra.
---------------xxx---------------
---------------------------------
You >> Canada?
AI >> Ottawa.
---------------xxx---------------
---------------------------------
You >> India?
AI >> Delhi.
---------------xxx---------------
Goodbye! Exiting chat...
